In [109]:
from scipy.optimize import minimize
import pandas as pd
import random
from PIL import Image, ImageDraw
from datetime import datetime
import time

In [110]:
def gen_alg(types, n, l, w, h, gen_num, max_epochs = 30, generation = None,
            absolute_entity = None):#stop_score
    
    best_absolute_score = 0
    start_time = datetime.now()
    for epoch in range(max_epochs):
        generation = new_generation(types, n, l, w, gen_num, generation)
        
        all_coeffs = [k['coeffs'] for k in generation]
        
        for entity_coefs in all_coeffs:
            entity_score = calculate_score(l, w, entity_coefs)
            if entity_score > best_absolute_score:
                absolute_entity = entity_coefs
                best_absolute_score = entity_score
                
        print('n :', n, 'epoch : ', epoch, )
        print("Время выполнения функции sum() в секундах:", print(datetime.now() - start_time))
        print(best_absolute_score)
    return absolute_entity, best_absolute_score


def generate_first_epoch(types, n, l, w, h, gen_num): #генерация первого замощения(0 эпоха)
    population = []
    for i in range(gen_num):
        coeffs = []
        for j in range(n):
            delta = random.randint(6,9)/10
            r = random.choice(types)
            coeffs.append(random.randint(0 + int(delta*r), l - int(delta*r)))
            coeffs.append(random.randint(0 + int(delta*r), w - int(delta*r)))
            coeffs.append(r)
        population.append(coeffs)
           
        
     
    probs = optimized_probs(l, w, population)
    print('zero generate complite!')

    return [{'prob': probs[i], 'coeffs': population[i]} for i in range(gen_num)]
    
    
    
def mutation_bias(types, l, w, coeffs): #возвращает коэффициенты, некоторые из которых заменяны на рандомные числа некоторой функции
    nums_of_mutations = random.randint(1, 1+len(coeffs)//5)
    for i in range(nums_of_mutations):
        index = random.randint(0, len(coeffs) - 1)

        if index%3 == 2:
            recall, effiency = draw_and_score(l,w,coeffs[::3],coeffs[1::3],coeffs[2::3])
            
            if recall < 0.65:
                ind_r = types.index(coeffs[index])
                
                if ind_r < len(types) - 1:
                    coeffs[index] = types[ind_r+1]
        else:
            coeffs[index] = abs(coeffs[index] - random.randint(-40, 40))
    return coeffs


def calculate_score(l, w, coeffs):
    x=coeffs[::3]
    y=coeffs[1::3]
    r=coeffs[2::3]
    recall, effiency = draw_and_score(l,w,x,y,r)
    f1 = 2*recall*effiency / (recall+ effiency)
    return f1


def euclidean_distance(p1, p2): #а зачем это надо..... а, для расчёта примлегого расстояния между 2 роутерами
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)


def crossover(a, b, index): #генерация потомка (коэффициентов)
    return [b['coeffs'][:index] + a['coeffs'][index:], a['coeffs'][:index] + b['coeffs'][index:]]


def who_will_survive(probs): #определение индекса выжившего потомка в паре 2 родителей
    return random.choices([i for i in range(len(probs))], probs)[0]
    

def optimized_probs(l, w, children):
    sum_error = 0
    errors = []
    
    for child in children:
        error = (1 - calculate_score(l, w, child))
        errors.append(error)
        sum_error+= 1 / error
        
    probs = []
    
    for i in range(len(children)):
        probs.append((1/errors[i])/sum_error)
    
    return probs
        

    
def new_generation(types, n, l, w, gen_num, old_generation = None): #генерация потомков и их мутация
    
    new_generation_genoms = []
    
    if old_generation == None:
        old_generation = generate_first_epoch(types, n, l, w, h, gen_num)
    
    if old_generation != None:
        for _ in range(gen_num):
            #кол-во родителей
            mother, father = random.choices(old_generation, [k['prob'] for k in old_generation], k = 2)

            children = []
            
            for i in range(len(mother['coeffs'])-1):
                children.extend(crossover(mother, father, i))
                
            childrens_probs = optimized_probs(l, w, children)
            
            index = who_will_survive(childrens_probs)
           
            new_generation_genoms.append(mutation_bias(types, l, w, children[index]))
            
        
        new_generation_probs = optimized_probs(l, w, new_generation_genoms)
        population = len(new_generation_genoms)
        
        return [{'prob': new_generation_probs[i], 'coeffs': new_generation_genoms[i]} for i in range(population)]

In [111]:
def draw_and_score(l,w,x,y,r):
    im = Image.new('RGB', (l, w), (255, 255, 255))
    draw = ImageDraw.Draw(im)
    
    absolute_max = 0
    
    for i in range(len(x)):
        draw.ellipse((x[i] - r[i], y[i] - r[i], x[i] + r[i], y[i] + r[i]), fill='red')
        absolute_max += 3.1415926 * r[i]**2
    im.save('draw.jpg', quality=100)
    
    im = Image.open("draw.jpg")
    pixels = im.load()  # список с пикселями
    x, y = im.size  # ширина (x) и высота (y) изображения
    
    white_pix = 0
    another_pix = 0

    for i in range(x):
        for j in range(y):

            color = pixels[i, j]  # содержит кортеж из нескольких значений цвета, в зависимости от формата изображения

            flag = True  # Флаг, является ли пиксель белым
            for q in range(3):
                # проверка чисто белых пикселей, для оттенков нужно использовать диапазоны
                if pixels[i, j][q] != 255:  # pixels[i, j][q] > 240  # для оттенков
                    flag = False

            if flag:
                white_pix += 1
            another_pix += 1
    
    recall = (100 - (round(white_pix / another_pix * 100, 3)))/100
    effiency = w*l*recall / absolute_max #precision
    
    return [recall, effiency]

In [112]:
l, w = 800, 600

n = 1
gen_num = 6
types = [70, 80, 90, 100, 110, 120]



samples = []
best_f1 = 0
best_sample = None

while n < 10:
    sample, f1 = gen_alg(types, n, l, w, h, gen_num, max_epochs = 5)
    if f1>best_f1:
        best_f1 = f1
        best_sample = sample
        
    samples[sample]
    n+=1

zero generate complite!
n : 1 epoch :  0
0:00:52.743685
Время выполнения функции sum() в секундах: None
0.18158592961748746
n : 1 epoch :  1
0:01:33.449761
Время выполнения функции sum() в секундах: None
0.18158592961748746


KeyboardInterrupt: 